# Assignment 1


In [1]:
import numpy as np
np.random.seed(13) #TODO Check if this is used for sgd
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Reshape, Lambda
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import model_to_dot
from keras.preprocessing import sequence
from gensim.models import KeyedVectors
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors as nn
from matplotlib import pylab
from __future__ import division

c:\users\illia\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
c:\users\illia\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# DO NOT Modify the lines in this cell
path = 'alice.txt'#'analogy_alice.txt'
corpus = open(path).readlines()[0:700]

corpus = [sentence for sentence in corpus if sentence.count(" ") >= 2]

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'+"'")
tokenizer.fit_on_texts(corpus)
corpus = tokenizer.texts_to_sequences(corpus)
nb_samples = sum(len(s) for s in corpus)
V = len(tokenizer.word_index) + 1

# Is this something they need to change?
dim = 100
window_size = 2 #use this window size for Skipgram, CBOW, and the model with the additional hidden layer
window_size_corpus = 4 #use this window size for the co-occurrence matrix

## Question 1

### Co-occurrence Matrix
Use the provided code to load the "Alice in Wonderland" text document. 
1. Implement the word-word co-occurrence matrix for “Alice in Wonderland”
2. Normalize the words such that every value lies within a range of 0 and 1
3. Compute the cosine distance between the given words:
    - Alice 
    - Dinah
    - Rabbit
4. List the 5 closest words to 'Alice'. Discuss the results.
5. Discuss what the main drawbacks are of a term-term co-occurence matrix solutions?


In [3]:

import codecs, scipy.sparse, numpy

#create co-occurrence matrix


#create_cooccurrence_matrix('alice.txt',corpus,window_size)
#print(corpus)
#coo=numpy.zeros((V, V))
#coo=[[0 for x in range(V)] for y in range(V)] 
vocabulary={}
data=[]
row=[]
col=[]
for sentence in corpus:
    for pos,word in enumerate(sentence):
        i=vocabulary.setdefault(word,len(vocabulary))
        start=max(0,pos-window_size_corpus)
        end=min(len(sentence),pos+window_size_corpus+1)
        for pos2 in range(start,end):
            if pos2==pos: 
                continue
            j=vocabulary.setdefault(sentence[pos2],len(vocabulary))
            if(j!=i):
                data.append(1.); row.append(i); col.append(j);
            
cooccurrence_matrix=scipy.sparse.coo_matrix((data,(row,col)))
#print(cooccurrence_matrix.T * cooccurrence_matrix)
print(cooccurrence_matrix.toarray())
#print(cooccurrence_matrix.toarray()[1180][0])
tokenizer.word_index
#The dog chased the cat away from the garden

[[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 8. ... 0. 0. 0.]
 [1. 8. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


{'the': 1,
 'and': 2,
 'to': 3,
 'she': 4,
 'a': 5,
 'i': 6,
 'it': 7,
 'of': 8,
 'was': 9,
 'in': 10,
 'alice': 11,
 'you': 12,
 'that': 13,
 'her': 14,
 'as': 15,
 'said': 16,
 'had': 17,
 'for': 18,
 'but': 19,
 'be': 20,
 'on': 21,
 'all': 22,
 'with': 23,
 'little': 24,
 'mouse': 25,
 'down': 26,
 'very': 27,
 'this': 28,
 'not': 29,
 'so': 30,
 'out': 31,
 'if': 32,
 'is': 33,
 'at': 34,
 't': 35,
 's': 36,
 'll': 37,
 'how': 38,
 'they': 39,
 'about': 40,
 'herself': 41,
 'me': 42,
 'up': 43,
 'what': 44,
 'way': 45,
 'when': 46,
 'like': 47,
 'one': 48,
 'do': 49,
 'no': 50,
 'oh': 51,
 'went': 52,
 'thought': 53,
 'again': 54,
 'there': 55,
 'see': 56,
 'or': 57,
 'could': 58,
 'would': 59,
 'think': 60,
 'them': 61,
 'know': 62,
 'rabbit': 63,
 'dear': 64,
 'were': 65,
 'time': 66,
 'get': 67,
 'here': 68,
 'must': 69,
 'my': 70,
 'by': 71,
 'into': 72,
 'found': 73,
 'such': 74,
 'began': 75,
 'soon': 76,
 'm': 77,
 'quite': 78,
 'then': 79,
 'off': 80,
 'now': 81,
 'go': 82

In [7]:
#find cosine similarity to Alice, Dinah and Rabbit
import math
def cos_sim(a,b):
    sum_c=0;len_c=len(a);sum_as=0;sum_bs=0;
    for ii in range(len_c):
        sum_c=sum_c+(a[ii]*b[ii])
        sum_as=sum_as+(a[ii]*a[ii])
        sum_bs=sum_bs+(b[ii]*b[ii])
    return sum_c/(math.sqrt(sum_as)*(math.sqrt(sum_bs)))


print(tokenizer.word_index['alice'])  #11
print(tokenizer.word_index['dinah'])  #87
print(tokenizer.word_index['rabbit']) #63





cooc=cooccurrence_matrix.toarray()
tf_cooc=numpy.zeros((V-1, V-1))
for i, sentence in enumerate(cooc):
    sumf=0
    for wordf in sentence:
        sumf+=wordf
    for j,wordf in enumerate(sentence):
        if(sumf>0):
            tf_cooc[i][j]=cooc[i][j]/sumf
        else:
            continue
#print(tf_cooc)

#print(len(cooc))
#print(len(cooc[10]))
#print(V)
#print(cooc[0])
#print(tokenizer.word_index)

from sklearn.metrics.pairwise import cosine_similarity
#print(len(tf_cooc[10:11][0]))
#print(len(cooc[V-2]))
#print(cooc.shape)
#print(tf_cooc.shape)
print(cosine_similarity(tf_cooc[10:11], tf_cooc[86:87]))
print(cos_sim(tf_cooc[10],tf_cooc[86]))                  
print(cosine_similarity(tf_cooc[10:11], tf_cooc[62:63]))
print(cos_sim(tf_cooc[10],tf_cooc[62]))
print(cosine_similarity(tf_cooc[86:87], tf_cooc[62:63]))
print(cos_sim(tf_cooc[86],tf_cooc[62]))



11
87
63
[[0.48967328]]
0.48967327730053256
[[0.08862668]]
0.08862667920586557
[[0.13969969]]
0.1396996890567782


In [20]:
#find the closest words to Alice
maxid=0
maxval=0
simi=np.zeros(V-1)
for i in range(V-1):
    simi[i]=cosine_similarity(tf_cooc[10:11], tf_cooc[i:i+1])[0][0]
indices=simi.argsort()[-6:][::-1]
print(indices)
print("closest word= {}, {}, {}, {}, {}".format(list(tokenizer.word_index)[indices[1]],list(tokenizer.word_index)[indices[2]],list(tokenizer.word_index)[indices[3]],list(tokenizer.word_index)[indices[4]],list(tokenizer.word_index)[indices[5]]))            
for i in range(1,6):
    print(cosine_similarity(tf_cooc[10:11], tf_cooc[indices[i]:indices[i]+1])[0][0])


[ 10  34  16 161  82   2]
closest word= t, had, question, say, to
0.7033898864296415
0.6995182846911174
0.6726422527548841
0.6685274041392557
0.6544770424322105


Discussion of the drawbacks:

In [29]:
#Save your all the vector representations of your word embeddings in this way
#Change when necessary the sizes of the vocabulary/embedding dimension

f = open('vectors_co_occurrence.txt',"w")
f.write(" ".join([str(V-1),str(V-1)]))
f.write("\n")

#vectors = your word co-occurrence matrix
vectors = cooccurrence_matrix.toarray()
for word, i in tokenizer.word_index.items():    
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, list(vectors[i-1,:]))))
    f.write("\n")
f.close()

In [4]:
#reopen your file as follows

co_occurrence = KeyedVectors.load_word2vec_format('./vectors_co_occurrence.txt', binary=False)

## Question 2

### Word embeddings
Build embeddings with a keras implementation where the embedding vector is of length 50, 150 and 300. Use the Alice in Wonderland text book for training.
1. Using the CBOW model
2. Using Skipgram model
3. Add extra hidden dense layer to CBow and Skipgram implementations. Choose an activation function for that layer and justify your answer.
4. Analyze the four different word embeddings
    - Implement your own function to perform the analogy task with. Do not use existing libraries for this task such as Gensim. Your function should be able to answer whether an anaology as in the example given in the pdf-file is true.
    - Compare the performance on the analogy task between the word embeddings that you have trained in 2.1, 2.2 and 2.3.  
    - Visualize your results and interpret your results
5. Use the word co-occurence matrix from Question 1. Compare the performance on the analogy task with the performance of your trained word embeddings.  
6. Discuss:
    - What are the main advantages of CBOW and Skipgram?
    - What is the advantage of negative sampling?
    - What are the main drawbacks of CBOW and Skipgram?
7. Load pre-trained embeddings on large corpuses (see the pdf file). You only have to consider the word embeddings with an embedding size of 300
    - Compare performance on the analogy task with your own trained embeddings from "Alice in Wonderland". You can limit yourself to the vocabulary of Alice in Wonderland. Visualize the pre-trained word embeddings and compare these with the results of your own trained word embeddings. 


In [3]:
#prepare data for cbow
import math
reversed_index = {v:k for k,v in tokenizer.word_index.items()}

def prep_cbow_data(corpus=corpus,window_size=window_size,V=V):
    flat = [word for line in corpus for word in line]
    vectors = []
    L = len(flat)
    start = int(window_size)
    x = []
    y = []
    for i in range(start,L-start):
        context_before = flat[i-start:i]
        target = np.zeros(V,dtype=int)
        target[flat[i]-1] = 1
        context_after = flat[i+1:i+1+start]
        context = context_before + context_after
        #onehot_context = []
        #for word in context:
        #    vector = np.zeros(V,dtype=int)
        #    vector[word-1] = 1
        #    onehot_context.append(vector)
            
        if len(context) == 4:
            #onehot_context = np.asarray(onehot_context)
            x.append(context)
            y.append(target)
        
    x = np.asarray(x)
    y = np.asarray(y)
    return x,y


prep_cbow_data()[0].shape

(6559, 4)

In [4]:
#create CBOW model
from keras.layers import Flatten
X,Y = prep_cbow_data()
features = len(X)
print("size X: {},size Y:{}".format(len(X),len(Y)))
print(X.shape,Y.shape)
cbow = Sequential()
cbow.add(Embedding(input_dim=V, output_dim=dim, init='glorot_uniform',input_length=4))
cbow.add(Flatten())
cbow.add(Dense(V, init='glorot_uniform', activation='softmax'))

size X: 6559,size Y:6559
(6559, 4) (6559, 1183)


c:\users\illia\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(input_dim=1183, output_dim=100, input_length=4, embeddings_initializer="glorot_uniform")`
  
c:\users\illia\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1183, activation="softmax", kernel_initializer="glorot_uniform")`
  # Remove the CWD from sys.path while we load stuff.


In [5]:
#define loss function
cbow.compile(loss='categorical_crossentropy', optimizer='adadelta')

In [6]:
#train model
from keras import callbacks
epochs=100
earlyStopping=callbacks.EarlyStopping(monitor='val_loss',min_delta=0.001, patience=0, verbose=0, mode='auto')

cbow.fit(X,Y,epochs=epochs,validation_split=0.1, callbacks=[earlyStopping])

Train on 5903 samples, validate on 656 samples
Epoch 1/100
5903/5903 [==============================] - 3s 547us/step - loss: 7.0495 - val_loss: 7.0279
Epoch 2/100
5903/5903 [==============================] - 3s 496us/step - loss: 6.9876 - val_loss: 6.9825
Epoch 3/100
5903/5903 [==============================] - 3s 485us/step - loss: 6.9145 - val_loss: 6.9132
Epoch 4/100
5903/5903 [==============================] - 3s 488us/step - loss: 6.8049 - val_loss: 6.7977
Epoch 5/100
5903/5903 [==============================] - 3s 480us/step - loss: 6.6370 - val_loss: 6.6216
Epoch 6/100
5903/5903 [==============================] - 3s 470us/step - loss: 6.4109 - val_loss: 6.4074
Epoch 7/100
5903/5903 [==============================] - 3s 453us/step - loss: 6.1725 - val_loss: 6.2239
Epoch 8/100
5903/5903 [==============================] - 3s 464us/step - loss: 5.9754 - val_loss: 6.0881
Epoch 9/100
5903/5903 [==============================] - 3s 457us/step - loss: 5.8161 - val_loss: 5.9828
Epoch 10

In [10]:
#prepare data for Skipgram
def generate_data_skipgram(corpus, window_size, V):
    maxlen = window_size*2
    all_in = []
    all_out = []
    for words in corpus:
        L = len(words)
        for index, word in enumerate(words):
            p = index - window_size
            n = index + window_size + 1
                    
            in_words = []
            labels = []
            for i in range(p, n):
                if i != index and 0 <= i < L:
                    in_words.append([word])
                    labels.append(words[i])
            if in_words != []:
                all_in.append(np.array(in_words,dtype=np.int32))
                all_out.append(np_utils.to_categorical(labels, V))
    return (all_in,all_out)

#get x and y's for data
x,y = generate_data_skipgram(corpus,window_size,V)

#save the preprocessed data of Skipgram
f = open('data_skipgram.txt' ,'w')

for input,outcome  in zip(x,y):
    input = np.concatenate(input)
    f.write(" ".join(map(str, list(input))))
    f.write(",")
    outcome = np.concatenate(outcome)
    f.write(" ".join(map(str,list(outcome))))
    f.write("\n")
f.close()

#load the preprocessed Skipgram data
def generate_data_skipgram_from_file():
    f = open('data_skipgram.txt' ,'r')
    for row in f:
        inputs,outputs = row.split(",")
        inputs = np.fromstring(inputs, dtype=int, sep=' ')
        inputs = np.asarray(np.split(inputs, len(inputs)))
        outputs = np.fromstring(outputs, dtype=float, sep=' ')
        outputs = np.asarray(np.split(outputs, len(inputs)))
        yield (inputs,outputs)

In [7]:
#create Skipgram model

dim1=50
dim2=150
dim3=300
skipgram_50 = Sequential()
skipgram_50.add(Embedding(input_dim=V, output_dim=dim1, embeddings_initializer='glorot_uniform', input_length=1))
skipgram_50.add(Reshape((dim1, )))
skipgram_50.add(Dense(input_dim=dim1, units=V, kernel_initializer='uniform', activation='softmax'))

skipgram_150 = Sequential()
skipgram_150.add(Embedding(input_dim=V, output_dim=dim2, embeddings_initializer='glorot_uniform', input_length=1))
skipgram_150.add(Reshape((dim2, )))
skipgram_150.add(Dense(input_dim=dim2, units=V, kernel_initializer='uniform', activation='softmax'))

skipgram_300 = Sequential()
skipgram_300.add(Embedding(input_dim=V, output_dim=dim3, embeddings_initializer='glorot_uniform', input_length=1))
skipgram_300.add(Reshape((dim3, )))
skipgram_300.add(Dense(input_dim=dim3, units=V, kernel_initializer='uniform', activation='softmax'))

In [8]:
#define loss function for Skipgram
skipgram_50.compile(loss='categorical_crossentropy', optimizer='adadelta')
skipgram_150.compile(loss='categorical_crossentropy', optimizer='adadelta')
skipgram_300.compile(loss='categorical_crossentropy', optimizer='adadelta')

In [11]:
#train Skipgram model
for ite in range(10):
    loss = 0.
    for x, y in generate_data_skipgram_from_file():
        loss += skipgram_50.train_on_batch(x, y)

    print(ite, loss)
    
f = open('vectors_skipgram_50.txt' ,'w')
f.write(" ".join([str(V-1),str(dim1)]))
f.write("\n")
vectors = skipgram_50.get_weights()[0]
for word, i in tokenizer.word_index.items():
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, list(vectors[i,:]))))
    f.write("\n")
f.close()

for ite in range(10):
    loss = 0.
    for x, y in generate_data_skipgram_from_file():
        loss += skipgram_150.train_on_batch(x, y)

    print(ite, loss)
    
f = open('vectors_skipgram_150.txt' ,'w')
f.write(" ".join([str(V-1),str(dim2)]))
f.write("\n")
vectors = skipgram_150.get_weights()[0]
for word, i in tokenizer.word_index.items():
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, list(vectors[i,:]))))
    f.write("\n")
f.close()

for ite in range(10):
    loss = 0.
    for x, y in generate_data_skipgram_from_file():
        loss += skipgram_300.train_on_batch(x, y)

    print(ite, loss)
    
f = open('vectors_skipgram_300.txt' ,'w')
f.write(" ".join([str(V-1),str(dim3)]))
f.write("\n")
vectors = skipgram_300.get_weights()[0]
for word, i in tokenizer.word_index.items():
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, list(vectors[i,:]))))
    f.write("\n")
f.close()

0 42019.0201215744
1 38379.62837553024
2 39003.5622587204
3 39509.36110877991
4 39705.85113596916
5 39885.63010597229
6 40046.92625975609
7 40210.36398935318
8 40391.53400826454
9 40586.17857551575
0 42003.86192703247
1 38329.2822098732
2 38850.55951523781
3 39214.83134794235
4 39390.2365026474
5 39541.21982002258
6 39695.80431985855
7 39864.555876255035
8 40036.627875089645
9 40212.43623352051
0 41972.32948112488
1 38263.37252306938
2 38669.966866493225
3 38963.96811294556
4 39128.82766032219
5 39276.19422340393
6 39422.713296175
7 39560.45543265343
8 39694.1005204916
9 39825.880415678024


In [8]:
#create CBOW model with additional dense layer
cbow2 = Sequential()
cbow2.add(Embedding(input_dim=V, output_dim=dim, init='glorot_uniform',input_length=4))
cbow2.add(Flatten())
cbow2.add(Dense(V,activation='relu'))
cbow2.add(Dense(V, init='glorot_uniform', activation='softmax'))

c:\users\illia\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(input_dim=1183, output_dim=100, input_length=4, embeddings_initializer="glorot_uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\illia\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1183, activation="softmax", kernel_initializer="glorot_uniform")`
  


In [10]:
#define loss function for CBOW + dense
cbow2.compile(loss='categorical_crossentropy', optimizer='adadelta')

In [11]:
#train model for CBOW + dense
epochs=100

earlyStopping=callbacks.EarlyStopping(monitor='val_loss',min_delta=0.001, patience=0, verbose=0, mode='auto')

cbow2.fit(X,Y,epochs=epochs,validation_split=0.1, callbacks=[earlyStopping])

Train on 5903 samples, validate on 656 samples
Epoch 1/100
5903/5903 [==============================] - 9s 2ms/step - loss: 6.7766 - val_loss: 6.1094
Epoch 2/100
5903/5903 [==============================] - 9s 1ms/step - loss: 5.8327 - val_loss: 5.7606
Epoch 3/100
5903/5903 [==============================] - 9s 2ms/step - loss: 5.5145 - val_loss: 5.5747
Epoch 4/100
5903/5903 [==============================] - 8s 1ms/step - loss: 5.2639 - val_loss: 5.4150
Epoch 5/100
5903/5903 [==============================] - 8s 1ms/step - loss: 5.0215 - val_loss: 5.2781
Epoch 6/100
5903/5903 [==============================] - 9s 2ms/step - loss: 4.7894 - val_loss: 5.1477
Epoch 7/100
5903/5903 [==============================] - 11s 2ms/step - loss: 4.5724 - val_loss: 5.0567
Epoch 8/100
5903/5903 [==============================] - 10s 2ms/step - loss: 4.3716 - val_loss: 4.9879
Epoch 9/100
5903/5903 [==============================] - 10s 2ms/step - loss: 4.1812 - val_loss: 4.9395
Epoch 10/100
5903/5903 

In [12]:
#create Skipgram with additional dense layer

#create Skipgram model

skipgram_50_dense = Sequential()
skipgram_50_dense.add(Embedding(input_dim=V, output_dim=dim1, embeddings_initializer='glorot_uniform', input_length=1))
skipgram_50_dense.add(Reshape((dim1, )))
skipgram_50_dense.add(Dense(V,activation='relu'))
skipgram_50_dense.add(Dense(input_dim=dim1, units=V, kernel_initializer='uniform', activation='softmax'))

skipgram_150_dense = Sequential()
skipgram_150_dense.add(Embedding(input_dim=V, output_dim=dim2, embeddings_initializer='glorot_uniform', input_length=1))
skipgram_150_dense.add(Reshape((dim2, )))
skipgram_150_dense.add(Dense(V,activation='relu'))
skipgram_150_dense.add(Dense(input_dim=dim2, units=V, kernel_initializer='uniform', activation='softmax'))

skipgram_300_dense = Sequential()
skipgram_300_dense.add(Embedding(input_dim=V, output_dim=dim3, embeddings_initializer='glorot_uniform', input_length=1))
skipgram_300_dense.add(Reshape((dim3, )))
skipgram_300_dense.add(Dense(V,activation='relu'))
skipgram_300_dense.add(Dense(input_dim=dim3, units=V, kernel_initializer='uniform', activation='softmax'))



#skipgram2 = Sequential()
#skipgram2.add(Embedding(input_dim=V, output_dim=dim, embeddings_initializer='glorot_uniform', input_length=1))
#skipgram2.add(Reshape((dim, )))
#skipgram2.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='softmax'))
#skipgram2.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='softmax'))

In [13]:
#define loss function for Skipgram + dense
skipgram_50_dense.compile(loss='categorical_crossentropy', optimizer='adadelta')
skipgram_150_dense.compile(loss='categorical_crossentropy', optimizer='adadelta')
skipgram_300_dense.compile(loss='categorical_crossentropy', optimizer='adadelta')

In [14]:
#train model for Skipgram + dense
#train Skipgram model
for ite in range(10):
    loss = 0.
    for x, y in generate_data_skipgram_from_file():
        loss += skipgram_50_dense.train_on_batch(x, y)

    print(ite, loss)
    
f = open('vectors_skipgram_50_dense.txt' ,'w')
f.write(" ".join([str(V-1),str(dim1)]))
f.write("\n")
vectors = skipgram_50_dense.get_weights()[0]
for word, i in tokenizer.word_index.items():
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, list(vectors[i,:]))))
    f.write("\n")
f.close()

for ite in range(10):
    loss = 0.
    for x, y in generate_data_skipgram_from_file():
        loss += skipgram_150_dense.train_on_batch(x, y)

    print(ite, loss)
    
f = open('vectors_skipgram_150_dense.txt' ,'w')
f.write(" ".join([str(V-1),str(dim2)]))
f.write("\n")
vectors = skipgram_150_dense.get_weights()[0]
for word, i in tokenizer.word_index.items():
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, list(vectors[i,:]))))
    f.write("\n")
f.close()

for ite in range(10):
    loss = 0.
    for x, y in generate_data_skipgram_from_file():
        loss += skipgram_300_dense.train_on_batch(x, y)

    print(ite, loss)
    
f = open('vectors_skipgram_300_dense.txt' ,'w')
f.write(" ".join([str(V-1),str(dim3)]))
f.write("\n")
vectors = skipgram_300_dense.get_weights()[0]
for word, i in tokenizer.word_index.items():
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, list(vectors[i,:]))))
    f.write("\n")
f.close()

0 39444.32987213135
1 38264.48951482773
2 37948.66480302811
3 37769.99622249603
4 37750.78943347931
5 37725.23108911514
6 37674.930666565895
7 37620.79466640949
8 37566.527002334595
9 37519.70512115955
0 39471.03422307968
1 38220.93018221855
2 37718.286794900894
3 37538.61131024361
4 37493.13153910637
5 37456.600872159004
6 37402.676347732544
7 37335.5792144537
8 37265.79830777645
9 37196.82523834705
0 39445.47623085976
1 38082.24569654465
2 37592.96987724304
3 37330.86567401886
4 37259.75267124176
5 37224.620362997055
6 37161.21744906902
7 37086.09090960026
8 37002.261269927025
9 36925.072355270386


In [ ]:
#save model
skipgram_50.save('skipgram_50.h5')
skipgram_150.save('skipgram_150.h5')
skipgram_300.save('skipgram_300.h5')
skipgram_50_dense.save('skipgram_50_dense.h5')
skipgram_150_dense.save('skipgram_150_dense.h5')
skipgram_300_dense.save('skipgram_300_dense.h5')

In [5]:
#TODO: Implement your own analogy function

key_cbow_50 ={}
key_cbow_150 ={}
key_cbow_300 ={}
key_cbow_50_dense ={}
key_cbow_150_dense ={}
key_cbow_300_dense ={}
key_skipgram_50 ={}
key_skipgram_150 ={}
key_skipgram_300 ={}
key_skipgram_50_dense ={}
key_skipgram_150_dense ={}
key_skipgram_300_dense ={}

embed_cbow_50 =[]
embed_cbow_150 =[]
embed_cbow_300 =[]
embed_cbow_50_dense =[]
embed_cbow_150_dense =[]
embed_cbow_300_dense =[]
embed_skipgram_50 =[]
embed_skipgram_150 =[]
embed_skipgram_300 =[]
embed_skipgram_50_dense =[]
embed_skipgram_150_dense =[]
embed_skipgram_300_dense =[]




def load_from_file(a):
    f = open(a ,'r')
    key_dict={}
    vect=[]
    for row in f:
        dimention=0
        inp = row.split("\n")[0].split(" ")
        if(len(inp)==2):
            dimention=inp[1]
        else:
            key_dict.update({inp[0]:len(key_dict)})
            vect.append(np.asarray(inp[1:],dtype=float))
    return (vect,key_dict)

(embed_cbow_50,key_cbow_50)=load_from_file('vectors_cbow_50.txt')
(embed_cbow_150,key_cbow_150)=load_from_file('vectors_cbow_150.txt')
(embed_cbow_300,key_cbow_300)=load_from_file('vectors_cbow_300.txt')
(embed_cbow_50_dense,key_cbow_50_dense)=load_from_file('vectors_cbow_50_dense.txt')
(embed_cbow_150_dense,key_cbow_150_dense)=load_from_file('vectors_cbow_150_dense.txt')
(embed_cbow_300_dense,key_cbow_300_dense)=load_from_file('vectors_cbow_300_dense.txt')
(embed_skipgram_50,key_skipgram_50)=load_from_file('vectors_skipgram_50.txt')
(embed_skipgram_150,key_skipgram_150)=load_from_file('vectors_skipgram_150.txt')
(embed_skipgram_300,key_skipgram_300)=load_from_file('vectors_skipgram_300.txt')
(embed_skipgram_50_dense,key_skipgram_50_dense)=load_from_file('vectors_skipgram_50_dense.txt')
(embed_skipgram_150_dense,key_skipgram_150_dense)=load_from_file('vectors_skipgram_150_dense.txt')
(embed_skipgram_300_dense,key_skipgram_300_dense)=load_from_file('vectors_skipgram_300_dense.txt')




In [41]:
import operator 
def load_analogy():
    f = open('analogy_alice.txt' ,'r')
    analogy_list=[]
    for row in f:
        a = row.split("\n")[0].split(" ")
        analogy_list.append(np.asarray(a))
    return analogy_list

def analogy_check(analogy_list,embed_matrix,dictionary):
    v=[[],[],[],[]]
    for i in range(4):
        if(analogy_list[i] in dictionary):
            v[i]=embed_matrix[dictionary[analogy_list[i]]-1]
        else:
            v[i]=np.zeros(150)
    #print(dictionary)
    #return abs(cos_sim(v[0],v[1])-cos_sim(v[2],v[3]))
    return cos_sim( list(map(operator.sub, list(map(sum, zip(v[0],v[1]))), v[2])),v[3])

analogy_list=load_analogy()
#print(analogy_list[0])
#analogy_check(["go","going","look","looking"],embed_skipgram_150, tokenizer.word_index )
analogy_check(["say","saying","sit","sitting"],cooccurrence_matrix.toarray(), tokenizer.word_index )
analogy_check(["sudden","suddenly","usual","usually"],embed_skipgram_300_dense, tokenizer.word_index )
#analogy_check(["fancy","fancying","alice","rabbit"],embed_skipgram_150, tokenizer.word_index )    

-0.08582410447347387

In [38]:
#read model from tim

from keras.models import load_model
cbow_50 = load_model('cbow50.h5')
cbow_150 = load_model('cbow150.h5')
cbow_300 = load_model('cbow300.h5')
cbow_50_dense = load_model('cbow_extended_50.h5')
cbow_150_dense = load_model('cbow_extended_150.h5')
cbow_300_dense = load_model('cbow_extended_300.h5')

f = open('vectors_cbow_50.txt' ,'w')
f.write(" ".join([str(V-1),str(dim1)]))
f.write("\n")
vectors = cbow_50.get_weights()[0]
for word, i in tokenizer.word_index.items():
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, list(vectors[i,:]))))
    f.write("\n")
f.close()

f = open('vectors_cbow_150.txt' ,'w')
f.write(" ".join([str(V-1),str(dim2)]))
f.write("\n")
vectors = cbow_150.get_weights()[0]
for word, i in tokenizer.word_index.items():
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, list(vectors[i,:]))))
    f.write("\n")
f.close()

f = open('vectors_cbow_300.txt' ,'w')
f.write(" ".join([str(V-1),str(dim3)]))
f.write("\n")
vectors = cbow_300.get_weights()[0]
for word, i in tokenizer.word_index.items():
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, list(vectors[i,:]))))
    f.write("\n")
f.close()

f = open('vectors_cbow_50_dense.txt' ,'w')
f.write(" ".join([str(V-1),str(dim1)]))
f.write("\n")
vectors = cbow_50_dense.get_weights()[0]
for word, i in tokenizer.word_index.items():
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, list(vectors[i,:]))))
    f.write("\n")
f.close()

f = open('vectors_cbow_150_dense.txt' ,'w')
f.write(" ".join([str(V-1),str(dim2)]))
f.write("\n")
vectors = cbow_150_dense.get_weights()[0]
for word, i in tokenizer.word_index.items():
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, list(vectors[i,:]))))
    f.write("\n")
f.close()

f = open('vectors_cbow_300_dense.txt' ,'w')
f.write(" ".join([str(V-1),str(dim3)]))
f.write("\n")
vectors = cbow_300_dense.get_weights()[0]
for word, i in tokenizer.word_index.items():
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, list(vectors[i,:]))))
    f.write("\n")
f.close()


Comparison performance:

In [ ]:
#Visualization results trained word embeddings
print()

Interpretation results of the visualization

Compare the results of the trained word embeddings with the word-word co-occurrence matrix

## Discussion of the advantages of CBOW and Skipgram, the advantages of negative sampling and drawbacks of CBOW and Skipgram

CBOW tries to predict a word by context, it sees the context and maximizes the probability of the target word.
This means CBOW is good at predicting frequent words. To train CBOW a reasonably low amount of data is sufficient.
The drawback of CBOW is that whilst it will preform well at predicting frequent words it will have low accuracy for less frequent words.
this is because some words compete in the sense that they are a valid target for the same context.
The more frequent word will then be predicted.

In negative sampling we choose random words to pair with the target and have an output of 0. 
The updating of the weights is then performed on these K samples, which reduces the computational requirements.
The model does not need all observations but simply only the K sampled pairs (context + target).

Skip gram is designed to predict context. It sees the target and tries to find the context around the word. 
Skip gram is rather well suited even for rare words. Thake the example delightfull, it will try to predict something like yesterday was a day.
Whilst if CBOw would have gotten this context delightfull would have never been predicted it would have chosen more frequent words like good.
The drawback for skip gram is that it requires a large amount of data to train. This is because if we for example look at the delightfull word it will have context.
Similar nice will also have a context, delightfull day and nice day are 2 independant sets. In cbow this use of nice and delightfull would be competing since the context day has both 
delightfull and nice as targets.

In [12]:
#load pretrained word embeddings of word2vec

path_word2vec = "tes\GoogleNews-vectors-negative300.bin"

word2vec = KeyedVectors.load_word2vec_format(path_word2vec, binary=True)

In [17]:
#load pretraind word embeddings of Glove
import gensim
#path = "tes\glove.6B\glove.6B.300d_converted.txt"
path = "tes\glove.6B.300d.txt"

#convert GloVe into word2vec format
gensim.scripts.glove2word2vec.get_glove_info(path)
gensim.scripts.glove2word2vec.glove2word2vec(path, "glove_converted.txt")

glove = KeyedVectors.load_word2vec_format("glove_converted.txt", binary=False)



In [39]:
#w2v and glove analogy performance
w2vanalogylist=[]
gloveanalogylist=[]
cbowanalogylist=[]
cbowdenseanalogylist=[]
skipgramanalogylist=[]
skipgramdenseanalogylist=[]

for analogy in analogy_list:
    if(analogy[0] in tokenizer.word_index and analogy[1] in tokenizer.word_index and analogy[2] in tokenizer.word_index and analogy[3] in tokenizer.word_index):
        try:
            w2vanalogy=word2vec.n_similarity([analogy[0],analogy[1]],[analogy[2],analogy[3]])
            gloveanalogy=glove.n_similarity([analogy[0],analogy[1]],[analogy[2],analogy[3]])
            cbowanalogy=analogy_check(analogy,embed_cbow_300, tokenizer.word_index )
            cbowdenseanalogy=analogy_check(analogy,embed_cbow_300_dense, tokenizer.word_index )
            skipgramanalogy=analogy_check(analogy,embed_skipgram_300, tokenizer.word_index )
            skipgramdenseanalogy=analogy_check(analogy,embed_skipgram_300_dense, tokenizer.word_index )
            
            print("w2v: {}, {}, {}, {}: {}".format(analogy[0],analogy[1],analogy[2],analogy[3],w2vanalogy))
            print("glove: {}, {}, {}, {}: {}".format(analogy[0],analogy[1],analogy[2],analogy[3],gloveanalogy))
            print("cbow: {}, {}, {}, {}: {}".format(analogy[0],analogy[1],analogy[2],analogy[3],cbowanalogy))
            print("cbow_dense: {}, {}, {}, {}: {}".format(analogy[0],analogy[1],analogy[2],analogy[3],cbowdenseanalogy))
            print("skipgram: {}, {}, {}, {}: {}".format(analogy[0],analogy[1],analogy[2],analogy[3],skipgramanalogy))
            print("skipgram_dense: {}, {}, {}, {}: {}".format(analogy[0],analogy[1],analogy[2],analogy[3],skipgramdenseanalogy))
            
            
            w2vanalogylist.append(w2vanalogy)
            gloveanalogylist.append(gloveanalogy)
            cbowanalogylist.append(cbowanalogy)
            cbowdenseanalogylist.append(cbowdenseanalogy)
            skipgramanalogylist.append(skipgramanalogy)
            skipgramdenseanalogylist.append(skipgramdenseanalogy)
            
            
            print()
        except KeyError:
            continue
    



w2v: sudden, suddenly, usual, usually: 0.22255026056670402
glove: sudden, suddenly, usual, usually: 0.3100777731068751
cbow: sudden, suddenly, usual, usually: -0.07675966522933991
cbow_dense: sudden, suddenly, usual, usually: 0.021816142041618355
skipgram: sudden, suddenly, usual, usually: 0.013981381463225495
skipgram_dense: sudden, suddenly, usual, usually: -0.08582410447347387

w2v: bad, worse, good, better: 0.6724906513772779
glove: bad, worse, good, better: 0.6565830036364912
cbow: bad, worse, good, better: -0.14211218867763775
cbow_dense: bad, worse, good, better: -0.06851437505368992
skipgram: bad, worse, good, better: -0.05229406240425803
skipgram_dense: bad, worse, good, better: 0.0010391543210957021

w2v: go, going, look, looking: 0.4437767197062708
glove: go, going, look, looking: 0.6695781772903449
cbow: go, going, look, looking: 0.0013563966233273222
cbow_dense: go, going, look, looking: 0.16969527138432886
skipgram: go, going, look, looking: 0.0857564110649774
skipgram_de

In [ ]:
#Visualize the pre-trained word embeddings
def draw_word_vecs(X,strings):
    col = ['r','b','g','y']
    if len(X)!= len(strings):
        print("mismatch in lengths between labels and vectors")
        
    X_embedded = TSNE(n_components=4,method='exact').fit_transform(X)
    maxt = np.amax(X_embedded)
    mint = np.amin(X_embedded)
    print(maxt,mint)
    maxt = max(maxt,mint*-1)
    for i in range(len(X_embedded)):
        plt.quiver(X_embedded[i][0]/maxt,X_embedded[i][1]/maxt,X_embedded[i][2]/maxt,X_embedded[i][3]/maxt,angles='xy', scale_units='xy',color=col[i], scale=1,label=strings[i])
        for val in X_embedded[i]:
            print(val/maxt)
    print(X_embedded)
    plt.ylim(-1.2,1.2)
    plt.xlim(-1.2,1.2)
    plt.show()
    


Comparison performance with your own trained word embeddings

In [22]:
tokenizer.word_index

{'the': 1,
 'and': 2,
 'to': 3,
 'she': 4,
 'a': 5,
 'i': 6,
 'it': 7,
 'of': 8,
 'was': 9,
 'in': 10,
 'alice': 11,
 'you': 12,
 'that': 13,
 'her': 14,
 'as': 15,
 'said': 16,
 'had': 17,
 'for': 18,
 'but': 19,
 'be': 20,
 'on': 21,
 'all': 22,
 'with': 23,
 'little': 24,
 'mouse': 25,
 'down': 26,
 'very': 27,
 'this': 28,
 'not': 29,
 'so': 30,
 'out': 31,
 'if': 32,
 'is': 33,
 'at': 34,
 't': 35,
 's': 36,
 'll': 37,
 'how': 38,
 'they': 39,
 'about': 40,
 'herself': 41,
 'me': 42,
 'up': 43,
 'what': 44,
 'way': 45,
 'when': 46,
 'like': 47,
 'one': 48,
 'do': 49,
 'no': 50,
 'oh': 51,
 'went': 52,
 'thought': 53,
 'again': 54,
 'there': 55,
 'see': 56,
 'or': 57,
 'could': 58,
 'would': 59,
 'think': 60,
 'them': 61,
 'know': 62,
 'rabbit': 63,
 'dear': 64,
 'were': 65,
 'time': 66,
 'get': 67,
 'here': 68,
 'must': 69,
 'my': 70,
 'by': 71,
 'into': 72,
 'found': 73,
 'such': 74,
 'began': 75,
 'soon': 76,
 'm': 77,
 'quite': 78,
 'then': 79,
 'off': 80,
 'now': 81,
 'go': 82

In [25]:
'pleasanter' in tokenizer.word_index

False